In [16]:
import numpy as np, pandas as pd
import glob
import os 
import json
import sys
import numpy
from math import log, log10
from gdpyc import GasMap, DustMap
from astropy.coordinates import SkyCoord
from astropy import units as u

df = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/demo/FGL_more/FGL_11152023_GammaCan.csv')

df = df[df['FGL'].str[-1]!='c']

df['GLON'] = SkyCoord(ra=df['ra'].values*u.degree, dec=df['dec'].values*u.degree, frame='icrs').galactic.l.degree
df['GLAT'] = SkyCoord(ra=df['ra'].values*u.degree, dec=df['dec'].values*u.degree, frame='icrs').galactic.b.degree

df['HR_ms'] = (df['Fcsc_m']-df['Fcsc_s'])/(df['Fcsc_m']+df['Fcsc_s'])
df['HR_hm'] = (df['Fcsc_h']-df['Fcsc_m'])/(df['Fcsc_h']+df['Fcsc_m'])
df = df.sort_values(by=['flux_aper90_ave_b'], ascending=False).reset_index(drop=True)

# print(df.columns)

df[['FGL','X_name','Class','name','flux_aper90_ave_b']]

df['name_nospace'] = df['X_name'].str.replace(' ','')#apply(lambda x: x.replace(' ', ''))



df_alls = pd.read_csv('/Users/huiyang/Research/GitHub/MUWCLASS_4FGL/codes/Classifications/data/Individual_sources/FGL_11152023_GammaCan_alls.csv')

# light curve >50 counts
# spectrum > 100 counts

df_select3 = df[df['X_name'].isin(df_alls.loc[df_alls['src_cnts_aper90_b']>=50, 'name'].unique())].reset_index(drop=True)

df_select3 = df_select3[~df_select3['X_name'].isin(['2CXO J174457.8-290509', '2CXO J174507.0-290357','2CXO J174506.8-290537','2CXO J111459.1-611707','2CXO J150234.5+015205','2CXO J073717.0+653557'])].reset_index(drop=True)
# print(df_select3)

coords = SkyCoord(df_select3['ra'], df_select3['dec'], unit='deg')
df_select3['ebv'] = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
df_select3['nH_from_AV'] = 2.21 * 3.1 * df_select3['ebv'] * 0.1
df_select3['nH']  = np.log10(GasMap.nh(coords, nhmap='LAB').value) #/ 1e22 # nH in unit of 1.e22 atoms /cm2

print(df_select3[['FGL','X_name','Class','CT','name','name_nospace','ra', 'dec','flux_aper90_ave_b','var_intra_prob',
       'var_inter_prob','nH']])

             FGL                 X_name    Class         CT  \
0   J0859.3-4342  2CXO J085927.0-434528       CV   2.451807   
1   J1725.1-3408  2CXO J172508.8-341112      YSO   0.335596   
2   J0859.2-4729  2CXO J085905.6-473041      YSO   4.286869   
3   J1742.5-2833  2CXO J174216.9-283707       CV   1.907402   
4   J1743.8-3143  2CXO J174347.2-314025  LM-STAR   4.563578   
..           ...                    ...      ...        ...   
56  J1510.9+0551  2CXO J151100.4+054921      AGN   3.911411   
57  J0859.3-4342  2CXO J085928.4-434628       NS   2.893635   
58  J1843.7-3227  2CXO J184332.8-322122      AGN   2.564143   
59  J1510.9+0551  2CXO J151051.3+054937      AGN  41.413040   
60  J1242.6+3236  2CXO J124235.1+323340      AGN  24.799075   

                       name          name_nospace          ra        dec  \
0   2CXO J085927.0-434528-1  2CXOJ085927.0-434528  134.862609 -43.757805   
1   2CXO J172508.8-341112-1  2CXOJ172508.8-341112  261.286915 -34.186814   
2   2CXO J08590

In [10]:
df.columns

Index(['P_AGN', 'P_CV', 'P_HM-STAR', 'P_HMXB', 'P_LM-STAR', 'P_LMXB', 'P_NS',
       'P_YSO', 'e_P_AGN', 'e_P_CV', 'e_P_HM-STAR', 'e_P_HMXB', 'e_P_LM-STAR',
       'e_P_LMXB', 'e_P_NS', 'e_P_YSO', 'Class', 'Class_prob', 'Class_prob_e',
       'name', 'CT', 'X_name', 'FGL', 'ra', 'dec', 'r0', 'significance',
       'Fcsc_s', 'e_Fcsc_s', 'Fcsc_m', 'e_Fcsc_m', 'Fcsc_h', 'e_Fcsc_h',
       'flux_aper90_ave_b', 'e_flux_aper90_ave_b', 'var_intra_prob',
       'var_inter_prob', 'MW_MW', 'MW_counts', 'match_flag', 'p_any', 'p_i',
       'Gmag', 'BPmag', 'RPmag', 'e_Gmag', 'e_BPmag', 'e_RPmag', 'RPlx', 'PM',
       'epsi', 'sepsi', 'ruwe', 'rgeo', 'Jmag', 'Hmag', 'Kmag', 'e_Jmag',
       'e_Hmag', 'e_Kmag', 'W1mag', 'W2mag', 'W3mag', 'e_W1mag', 'e_W2mag',
       'e_W3mag', 'r_hat', 'X_PA', 'G_ra', 'G_dec', 'G_aF', 'G_bF', 'G_PA',
       'sep', 'Fb', 'HR_hms', 'popup', 'shape', 'color', 'GLON', 'GLAT',
       'name_nospace'],
      dtype='object')

In [44]:
df_srcs = pd.DataFrame()


for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    # if src != '2CXOJ174457.8-290509':
    #     continue
    #print(src)
    # if index >40:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    
    src_stat = {'FGL':df_src['FGL'], 'name': src_short,'l':df_src['GLON'],'b':df_src['GLAT'], 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH'], 'var_inter':df_src['var_inter_prob'], 'var_intra':df_src['var_intra_prob']}
    # coords = SkyCoord(ra, dec, unit='deg')
    # ebv = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
    # nH_from_AV = df_src['nH_from_AV']# 2.21 * 3.1 * ebv * 0.1
    # nH  = df_src['nH']# GasMap.nh(coords, nhmap='LAB').value / 1e22 # nH in unit of 1.e22 atoms /cm2
    #print(f'nH: {nH} 1.e22 atoms /cm2')

    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']

    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')

    dir_abs = f'/Users/huiyang/Research/GitHub/FGL_spectral_summary/bxa_results/{src}/'

    
    # os.system(f'python model_compare.py {dir_abs}powerlaw_out_/ {dir_abs}mekal_out_/ {dir_abs}bb_out_/  > {src}_model_compare.txt')

    prefixes = [f'{dir_abs}powerlaw_out_/',  f'{dir_abs}mekal_out_/', f'{dir_abs}bb_out_/']
    models = dict([(f, json.load(open(f + "info/results.json"))['logz']) for f in prefixes])

    best = max(models, key=models.__getitem__)
    Zbest = models[best]
    for m in models: models[m] -= Zbest
    Ztotal = log(sum(numpy.exp([Z for Z in list(models.values())])))
    limit = 30 # for example, Jeffreys scale for the Bayes factor

    # print()
    # print('Model comparison')
    # print('****************')
    # print()



    for m, md, md_f,p_n,p_l_n,p_u_n in zip(models, ['pl', 'mk', 'bb'], ['powerlaw','mekal','bb'], ['pl1.PhoIndex_median','src1.logkT_median','src1.logkT_median'], ['pl1.PhoIndex_errlo','src1.logkT_errlo','src1.logkT_errlo'], ['pl1.PhoIndex_errup','src1.logkT_errup','src1.logkT_errup']):
        df_src_sum = pd.read_csv(f'{dir_abs}{md_f}_out_/info/post_summary.csv')
        # print(df_src_sum.columns)
        src_stat[md],src_stat[md+'_l'],src_stat[md+'_u'] = df_src_sum.loc[0, p_n],df_src_sum.loc[0, p_l_n],df_src_sum.loc[0, p_u_n]
        Zrel = models[m]
        src_stat['log10Z_'+md] = Zrel / log(10)
        if Zrel == 0:
            src_stat['nH'],src_stat['nH_l'],src_stat['nH_u'] = df_src_sum.loc[0, 'src1.lognH_median'],df_src_sum.loc[0, 'src1.lognH_errlo'],df_src_sum.loc[0, 'src1.lognH_errup']
    df_sr = pd.DataFrame(data=src_stat, index=[0])
    df_srcs = pd.concat([df_srcs, df_sr], ignore_index=True, sort=False)
    # for md in ['powerlaw','mekal','bb']:

    print('  <tr>')
    print(f"   <td><a href='./srcs/{src}.html'>{src_stat['name']}</a></td>")
    print(f"   <td>{src_stat['FGL']}</td>")
    print(f"   <td>{src_stat['l']:.7f}</td>")
    print(f"   <td>{src_stat['b']:.7f}</td>")
    print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
    print(f"   <td>{src_stat['CT']:.2f}</td>")
    print(f"   <td>{src_stat['fb']:.2e}</td>")
    print(f"   <td>{src_stat['var_inter']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_lc_500s.png" target="_blank">{src_stat["var_intra"]:.2f}</a></td>')
    print(f"   <td>{src_stat['nobs']:.1f}</td>")
    print(f"   <td>{src_stat['ncounts']:.1f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}<sup>+{src_stat["pl_u"]-src_stat["pl"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{src_stat["pl"]-src_stat["pl_l"]:.2}</sub></a></td>')
    # print(f"   <td>{src_stat['log10Z_pl']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_pl_corner.pdf" target="_blank">{src_stat["log10Z_pl"]:.2f}</a></td>')
    print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}<sup>+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</sub></a></td>')
    print(f'   <td><a class="image-link" href="./images/{src}_mk_corner.pdf" target="_blank">{src_stat["log10Z_mk"]:.2f}</a></td>')
    # print(f"   <td>{src_stat['log10Z_mk']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}<sup>+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</sub></a></td>')
    # print(f"   <td>{src_stat['log10Z_bb']:.2f}</td>")
    print(f'   <td><a class="image-link" href="./images/{src}_bb_corner.pdf" target="_blank">{src_stat["log10Z_bb"]:.2f}</a></td>')
    print(f"   <td>{src_stat['nH']:.2f}<sup>+{src_stat['nH_u']-src_stat['nH']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['nH']-src_stat['nH_l']:.2}</sub></td>")
    print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
    print('  </tr>')


    

  <tr>
   <td><a href='./srcs/2CXOJ085927.0-434528.html'>J085927.0-434528</a></td>
   <td>J0859.3-4342</td>
   <td>265.1473173</td>
   <td>1.4499867</td>
   <td><a class="image-link" href="./images/shap/J085927.0-434528_CV.png" target="_blank">CV</a></td>
   <td>2.45</td>
   <td>1.59e-12</td>
   <td>nan</td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_lc_500s.png" target="_blank">1.00</a></td>
   <td>1.0</td>
   <td>6082.5</td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_pl_spectrum.png" target="_blank">1.63<sup>+0.06</sup><sub style="position: relative; right: +2.3em;">-0.054</sub></a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_pl_corner.pdf" target="_blank">-10.45</a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-434528_mk_spectrum.png" target="_blank">7.82<sup>+0.81</sup><sub style="position: relative; right: +2.3em;">-0.66</sub></a></td>
   <td><a class="image-link" href="./images/2CXOJ085927.0-43

In [6]:
df_srcs.to_csv('FGL_11152023_GammaCan_summary.csv',index=False)

In [43]:
df_srcs = pd.DataFrame()


for index, df_src in df_select3.iterrows():# ['name_nospace'].values:
    
    src = df_src['name_nospace']
    src_short = src[4:]
    # if src != '2CXOJ172508.8-341112':
    #     continue
    #print(src)
    # if index >0:
    #     continue

    # df_src = df_select[df_select['name_nospace']==src]
    ra, dec, src_name = df_src['ra'], df_src['dec'], df_src['X_name']
    # print(src_name, ra, dec)
    
    src_stat = {'FGL':df_src['FGL'], 'name': src_short,'l':df_src['GLON'],'b':df_src['GLAT'], 'Class':df_src['Class'], 'CT':df_src['CT'], 'nH_LAB':df_src['nH'], 'var_inter':df_src['var_inter_prob'], 'var_intra':df_src['var_intra_prob']}
    # coords = SkyCoord(ra, dec, unit='deg')
    # ebv = DustMap.ebv(coords, dustmap='SFD') * 0.86 # 0.86 is the correction described in Schlafly et al. 2010 and Schlafly & Finkbeiner 2011
    # nH_from_AV = df_src['nH_from_AV']# 2.21 * 3.1 * ebv * 0.1
    # nH  = df_src['nH']# GasMap.nh(coords, nhmap='LAB').value / 1e22 # nH in unit of 1.e22 atoms /cm2
    #print(f'nH: {nH} 1.e22 atoms /cm2')

    # df_var = search_file.table.to_pandas()
    df_var = df_alls[(df_alls['name']==src_name) & (df_alls['src_cnts_aper90_b']>=50)].reset_index(drop=True)
    # print(df_var[['name','obsid','obi','region_id','src_cnts_aper90_b']])
    # print(df_var['powlaw_nh'][0]/100,nH_from_AV,nH)
    
    # '''
    df_var['obsid'] = df_var['obsid'].apply(lambda x: str(x).zfill(5))

    src_stat['nobs'] = len(df_var)
    src_stat['ncounts'] = df_var['src_cnts_aper90_b'].sum()
    src_stat['fb'] = df_src['flux_aper90_ave_b']

    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md twomekal')
    # os.system(f'python customfitter_wstat_logpar.py -fn {src}_files.txt -md powerlawbb')

    dir_abs = f'/Users/huiyang/Research/GitHub/FGL_spectral_summary/bxa_results/{src}/'

    
    # os.system(f'python model_compare.py {dir_abs}powerlaw_out_/ {dir_abs}mekal_out_/ {dir_abs}bb_out_/  > {src}_model_compare.txt')

    prefixes = [f'{dir_abs}powerlaw_out_/',  f'{dir_abs}mekal_out_/', f'{dir_abs}bb_out_/']
    models = dict([(f, json.load(open(f + "info/results.json"))['logz']) for f in prefixes])

    best = max(models, key=models.__getitem__)
    Zbest = models[best]
    for m in models: models[m] -= Zbest
    Ztotal = log(sum(numpy.exp([Z for Z in list(models.values())])))
    limit = 30 # for example, Jeffreys scale for the Bayes factor

    # print()
    # print('Model comparison')
    # print('****************')
    # print()



    for m, md, md_f,p_n,p_l_n,p_u_n in zip(models, ['pl', 'mk', 'bb'], ['powerlaw','mekal','bb'], ['pl1.PhoIndex_median','src1.logkT_median','src1.logkT_median'], ['pl1.PhoIndex_errlo','src1.logkT_errlo','src1.logkT_errlo'], ['pl1.PhoIndex_errup','src1.logkT_errup','src1.logkT_errup']):
        df_src_sum = pd.read_csv(f'{dir_abs}{md_f}_out_/info/post_summary.csv')
        # print(df_src_sum.columns)
        src_stat[md],src_stat[md+'_l'],src_stat[md+'_u'] = df_src_sum.loc[0, p_n],df_src_sum.loc[0, p_l_n],df_src_sum.loc[0, p_u_n]
        Zrel = models[m]
        src_stat['log10Z_'+md] = Zrel / log(10)
        if Zrel == 0:
            src_stat['nH'],src_stat['nH_l'],src_stat['nH_u'] = df_src_sum.loc[0, 'src1.lognH_median'],df_src_sum.loc[0, 'src1.lognH_errlo'],df_src_sum.loc[0, 'src1.lognH_errup']
    df_sr = pd.DataFrame(data=src_stat, index=[0])
    df_srcs = pd.concat([df_srcs, df_sr], ignore_index=True, sort=False)
    # for md in ['powerlaw','mekal','bb']:

    temp_html = open('./srcs/template.html','r')

    lines = temp_html.readlines()

    lines[29] = lines[29].replace('2CXO J085927.0-434528', src_name)
    lines[65] = lines[65].replace('134.862609 -43.757805', f"{df_src['ra']}, {df_src['dec']}")

    lines[224] = lines[224].replace('J085927.0-434528',src)
    lines[225] = lines[225].replace('J0859.3-4342',df_src['FGL'])
    lines[226] = lines[226].replace('265.1473173',f"{src_stat['l']:.7f}")
    lines[227] = lines[227].replace('1.4499867',f"{src_stat['b']:.7f}")
    lines[228] = lines[228].replace('<td><a class="image-link" href="../images/shap/J085927.0-434528_CV.png" target="_blank">CV</a></td>',f'<td><a class="image-link" href="../images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
    lines[229] = lines[229].replace('0.658442',f"{df_src['Class_prob']:.3f}")
    lines[230] = lines[230].replace('0.108274',f"{df_src['Class_prob_e']:.3f}")
    lines[231] = lines[231].replace('2.45',f"{df_src['CT']:.2}")
    
    lines[255] = lines[255].replace('1.59e-12',f"{df_src['flux_aper90_ave_b']:.2e}")
    lines[256] = lines[256].replace('8.90e-15',f"{df_src['Fcsc_s']:.2e}")
    lines[257] = lines[257].replace('1.10e-13',f"{df_src['Fcsc_m']:.2e}")
    lines[258] = lines[258].replace('1.47e-12',f"{df_src['Fcsc_h']:.2e}")
    lines[259] = lines[259].replace('0.84965',f"{df_src['HR_hms']:.2f}")
    lines[260] = lines[260].replace('0.850961',f"{df_src['HR_ms']:.2f}")
    lines[261] = lines[261].replace('0.86007',f"{df_src['HR_hm']:.2f}")
    lines[262] = lines[262].replace('nan',f"{df_src['var_inter_prob']:.2f}")
    lines[263] = lines[263].replace('<a class="image-link" href="../images/2CXOJ085927.0-434528_lc_500s.png" target="_blank">1.00</a>',f'<a class="image-link" href="../images/{src}_lc_500s.png" target="_blank">{src_stat["var_intra"]:.2f}</a>')
    lines[264] = lines[264].replace('1.0',f"{src_stat['nobs']:.1f}")
    lines[265] = lines[265].replace('6082.5',f"{src_stat['ncounts']:.0f}")

    lines[292] = lines[292].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_pl_spectrum.png" target="_blank">3.61</a>',f'<a class="image-link" href="../images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}</a>')
    lines[293] = lines[293].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_pl_corner.pdf" target="_blank">-6.37</a>',f'<a class="image-link" href="../images/{src}_pl_corner.pdf" target="_blank">{src_stat["log10Z_pl"]:.2f}</a>')
    lines[294] = lines[294].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_mk_spectrum.png" target="_blank">1.28</a>',f'<a class="image-link" href="../images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}</a>')
    lines[295] = lines[295].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_mk_corner.pdf" target="_blank">0.00</a>',f'<a class="image-link" href="../images/{src}_mk_corner.pdf" target="_blank">{src_stat["log10Z_mk"]:.2f}</a>')
    lines[296] = lines[296].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_bb_spectrum.png" target="_blank">0.59</a>',f'<a class="image-link" href="../images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}</a>')
    lines[297] = lines[297].replace('<a class="image-link" href="../images/2CXOJ085905.6-473041_bb_corner.pdf" target="_blank">-47.39</a>',f'<a class="image-link" href="../images/{src}_bb_corner.pdf" target="_blank">{src_stat["log10Z_bb"]:.2f}</a>')
    lines[298] = lines[298].replace('22.42',f"{src_stat['nH']:.2f}")
    lines[299] = lines[299].replace('21.81',f"{src_stat['nH_LAB']:.2f}")

    lines[321] = lines[321].replace('18.130289',f"{df_src['Gmag']:.3f}")
    lines[322] = lines[322].replace('20.997097',f"{df_src['BPmag']:.3f}")
    lines[323] = lines[323].replace('16.169699',f"{df_src['RPmag']:.3f}")
    lines[324] = lines[324].replace('<td></td>',f"<td>{df_src['Jmag']:.3f}</td>")
    lines[325] = lines[325].replace('<td></td>',f"<td>{df_src['Hmag']:.3f}</td>")
    lines[326] = lines[326].replace('<td></td>',f"<td>{df_src['Kmag']:.3f}</td>")
    lines[327] = lines[327].replace('<td></td>',f"<td>{df_src['W1mag']:.3f}</td>")
    lines[328] = lines[328].replace('<td></td>',f"<td>{df_src['W2mag']:.3f}</td>")
    lines[329] = lines[329].replace('1580.09277',f"{df_src['rgeo']:.1f}")
    lines[330] = lines[330].replace('0.009257',f"{df_src['PM']:.5f}")


    src_html = open(f'./srcs/{src}.html','w')
    
    src_html.writelines((lines))
    src_html.close()

    # count = 0
    # for line in lines:
    #     count +=1
    #     if count == 66:
    #         print(line)

    # print('  <tr>')
    # print(f"   <td>{src_stat['name']}</td>")
    # print(f"   <td>{src_stat['FGL']}</td>")
    # print(f"   <td>{src_stat['l']:.7f}</td>")
    # print(f"   <td>{src_stat['b']:.7f}</td>")
    # print(f'   <td><a class="image-link" href="./images/shap/{src[4:]}_{src_stat["Class"]}.png" target="_blank">{src_stat["Class"]}</a></td>')
    # print(f"   <td>{src_stat['CT']:.2f}</td>")
    # print(f"   <td>{src_stat['fb']:.2e}</td>")
    # print(f"   <td>{src_stat['var_inter']:.2f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_lc_500s.png" target="_blank">{src_stat["var_intra"]:.2f}</a></td>')
    # print(f"   <td>{src_stat['nobs']:.1f}</td>")
    # print(f"   <td>{src_stat['ncounts']:.1f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_pl_spectrum.png" target="_blank">{src_stat["pl"]:.2f}<sup>+{src_stat["pl_u"]-src_stat["pl"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{src_stat["pl"]-src_stat["pl_l"]:.2}</sub></a></td>')
    # # print(f"   <td>{src_stat['log10Z_pl']:.2f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_pl_corner.pdf" target="_blank">{src_stat["log10Z_pl"]:.2f}</a></td>')
    # print(f'   <td><a class="image-link" href="./images/{src}_mk_spectrum.png" target="_blank">{10**src_stat["mk"]:.2f}<sup>+{10**src_stat["mk_u"]-10**src_stat["mk"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["mk"]-10**src_stat["mk_l"]:.2}</sub></a></td>')
    # print(f'   <td><a class="image-link" href="./images/{src}_mk_corner.pdf" target="_blank">{src_stat["log10Z_mk"]:.2f}</a></td>')
    # # print(f"   <td>{src_stat['log10Z_mk']:.2f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_bb_spectrum.png" target="_blank">{10**src_stat["bb"]:.2f}<sup>+{10**src_stat["bb_u"]-10**src_stat["bb"]:.2f}</sup><sub style="position: relative; right: +2.3em;">-{10**src_stat["bb"]-10**src_stat["bb_l"]:.2}</sub></a></td>')
    # # print(f"   <td>{src_stat['log10Z_bb']:.2f}</td>")
    # print(f'   <td><a class="image-link" href="./images/{src}_bb_corner.pdf" target="_blank">{src_stat["log10Z_bb"]:.2f}</a></td>')
    # print(f"   <td>{src_stat['nH']:.2f}<sup>+{src_stat['nH_u']-src_stat['nH']:.2f}</sup><sub style='position: relative; right: +2.3em;'>-{src_stat['nH']-src_stat['nH_l']:.2}</sub></td>")
    # print(f"   <td>{src_stat['nH_LAB']:.2f}</td>")
    # print('  </tr>')


    

In [28]:
df_select3.columns

df_select3.loc[df_select3['X_name']=='2CXO J085927.0-434528', ['FGL','X_name',#'ra','dec','GLON', 'GLAT','Class','Class_prob','Class_prob_e','CT',]]#
# 'flux_aper90_ave_b','Fcsc_s','Fcsc_m','Fcsc_h','var_intra_prob','var_inter_prob','HR_hms','HR_ms','HR_hm']]#
 'Gmag', 'BPmag', 'RPmag','Jmag', 'Hmag', 'Kmag','W1mag', 'W2mag','rgeo','PM',]]

,FGL,X_name,Gmag,BPmag,RPmag,Jmag,Hmag,Kmag,W1mag,W2mag,rgeo,PM
0,J0859.3-4342,2CXO J085927.0-434528,18.130289,20.997097,16.169699,NaN,NaN,NaN,NaN,NaN,1580.09277,0.009257
